<a href="https://colab.research.google.com/github/DaryaKV/DA_Python_Hometasks/blob/main/onlineShopping_c(%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5%202).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В файле <i>orders</i> представлена информация о заказах в онлайн-магазине.
```
OrderID - ID Заказа,
UserID - ID клиента,
OrderTime - время заказа,
items - список элементов в заказе, каждый элемент описан словарем:
    ItemID  - ID продукта,
    price - цена единицы товара,
    quantity - количество единиц товара в заказе
```
В файле <i>categories</i> представлена информация о категории товаров.
```
ItemID - ID продукта,
Category - категория продукта,
ParentCategory - более крупная категория продукта
```

## Анализ логов продуктового магазина

Представьте, что вы работаете аналитиком онлайн-магазина и ваша цель - повысить GMV (суммарную выручку от продажи товаров). Для этого вы хотите провести ряд исследований и придумать какие-то меры для увеличения выручки.

* Определите, какого числа было сделано больше всего заказов?
* Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)
* Напишите функцию, которая принимает на вход OrderID и рекомендует ItemID продукта для корзины этого заказа. Проверьте работу этой функции.

Казанцева Дарья Вячеславовна_Школа DA

In [2]:
import pandas as pd

In [4]:
categories = pd.read_csv('categories.csv')
orders = pd.read_json('orders.json')

In [ ]:
categories.head()

,ItemID,Category,ParentCategory
0,1000002,Сладости для диабетиков,Сладости
1,1000046,Вода,Напитки
2,1000048,Сладости для диабетиков,Сладости
3,1000049,Вода,Напитки
4,1000061,Фрукты,Овощи и фрукты


In [ ]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35030 entries, 0 to 35029
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ItemID          35030 non-null  int64 
 1   Category        35030 non-null  object
 2   ParentCategory  35030 non-null  object
dtypes: int64(1), object(2)
memory usage: 821.1+ KB


In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40375 entries, 0 to 40374
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   OrderID    40375 non-null  int64         
 1   UserID     40375 non-null  float64       
 2   OrderTime  40375 non-null  object        
 3   items      40375 non-null  object        
 4   Date       40375 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 1.8+ MB


# Какого числа было сделано больше всего заказов?

In [5]:
orders.groupby('Date')['OrderID'].count().head(1)


,OrderID
Date,
2021-01-01,75


# Найдите топ-3 ParentCategory по GMV (суммарная выручка, которую принесла продажа товаров этой категории)

In [6]:
items = orders['items'][1]
items = pd.DataFrame(items)

In [7]:
for i in items:
  cost = items['price'] * items['quantity']

items['cost'] = cost

In [8]:
items.cost = items.cost.astype(float)

In [9]:
items.ItemID = items.ItemID.astype(int)

In [10]:
grouped_items = items.groupby(['ItemID'])['cost'].sum()
grouped_items = pd.DataFrame(grouped_items)

In [11]:
grouped_items.reset_index().set_index('ItemID')

,cost
ItemID,
303673,179.70
303686,321.90
306115,53.26
308364,159.90
314641,527.52
316064,39.90
325893,91.80
417849,248.30
434637,38.90


In [12]:
grouped_items_parent = grouped_items.join(categories.set_index('ItemID'),how='left')

In [13]:
grouped_items_parent.groupby(['ParentCategory'])['cost'].sum().sort_values(ascending=[False]).head(3)

,cost
ParentCategory,
Молочная продукция,3012.68
"Мясо, птица, рыба",2109.48
Овощи и фрукты,1029.12


# Напишите функцию, которая принимает на вход OrderID и рекомендует ItemID продукта для корзины этого заказа. Проверьте работу этой функции.

In [ ]:
for i in range(orders.shape[0]):
  orders['ItemID'][i] = orders.loc[i,'items'][0]['ItemID']

In [25]:
orders

,OrderID,UserID,OrderTime,items,Date,ItemID
0,90659865,1.609460e+18,2021-01-01 03:51:42,"[{'ItemID': '413736', 'price': 36.72, 'quantit...",2021-01-01,413736
1,90622054,2.499293e+18,2021-01-01 04:13:13,"[{'ItemID': '961454', 'price': 179.9, 'quantit...",2021-01-01,961454
2,90659952,3.945866e+17,2021-01-01 05:48:53,"[{'ItemID': '306553', 'price': 101.9, 'quantit...",2021-01-01,306553
3,89765281,5.412765e+18,2021-01-01 08:33:41,"[{'ItemID': '979703', 'price': 99.9, 'quantity...",2021-01-01,979703
4,90582308,6.380061e+18,2021-01-01 09:55:35,"[{'ItemID': '303765', 'price': 44.9, 'quantity...",2021-01-01,303765
...,...,...,...,...,...,...
40370,93920417,8.412061e+18,2021-06-30 23:40:23,"[{'ItemID': '310971', 'price': 77.49, 'quantit...",2021-06-30,310971
40371,93924602,7.464423e+18,2021-06-30 23:41:41,"[{'ItemID': '509361', 'price': 79.9, 'quantity...",2021-06-30,509361
40372,93952166,1.605989e+18,2021-06-30 23:49:12,"[{'ItemID': '444839', 'price': 129.9, 'quantit...",2021-06-30,444839
40373,93955735,7.254769e+18,2021-06-30 23:56:21,"[{'ItemID': '875741', 'price': 49.0, 'quantity...",2021-06-30,875741


In [98]:
def Recommend(input_order_id):
  order_ind = orders.index[orders.OrderID == input_order_id][0]
  ItemID_of_order = orders.ItemID[order_ind]
  ItemID_of_order = int(ItemID_of_order)
  Category_search = categories.Category[categories.ItemID == ItemID_of_order].values[0]
  print(f'Интересуют товары из категории {Category_search}? Предлагаем обратить внимание на следующие позиции:{categories.ItemID[categories.Category == Category_search].head(5).values}')

Recommend(input())

Интересуют товары из категории Творог, сырки? Предлагаем обратить внимание на следующие позиции:[1000539 1000762 1000763 1001023 1001844]
